In [1]:
import tensorflow as tf
import os
import numpy as np
import pandas as pd

D:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## 导入数据

In [2]:
TRAIN_URL = "http://download.tensorflow.org/data/iris_training.csv"
TEST_URL =  "http://download.tensorflow.org/data/iris_test.csv"

CSV_COLUMN_NAMES =  ['SepalLength', 'SepalWidth',
                    'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

def load_data(label_name='Species'):
    #训练集
    train_path = tf.keras.utils.get_file(fname=os.path.basename(TRAIN_URL),
                                         origin=TRAIN_URL)
    train = pd.read_csv(train_path,names=CSV_COLUMN_NAMES,header=0)
    
    train_features,train_labels = train,train.pop(label_name)
    #测试集
    test_path = tf.keras.utils.get_file(fname=os.path.basename(TEST_URL),
                                       origin=TEST_URL)
    test = pd.read_csv(test_path,names=CSV_COLUMN_NAMES,header=0)
    
    test_features,test_labels = test,test.pop(label_name)
    
    return (train_features,train_labels),(test_features,test_labels)

In [3]:
(train_x,train_y),(test_x,test_y) = load_data()
print(train_x[:5])
print(train_y[:5])
print(type(train_x))

   SepalLength  SepalWidth  PetalLength  PetalWidth
0          6.4         2.8          5.6         2.2
1          5.0         2.3          3.3         1.0
2          4.9         2.5          4.5         1.7
3          4.9         3.1          1.5         0.1
4          5.7         3.8          1.7         0.3
0    2
1    1
2    2
3    0
4    0
Name: Species, dtype: int64
<class 'pandas.core.frame.DataFrame'>


## 特征列

In [4]:
feature_columns = []
print(train_x.keys())
for key in train_x.keys():
    feature_columns.append(tf.feature_column.numeric_column(key=key)) 
print(feature_columns)

Index(['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth'], dtype='object')
[_NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), _NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), _NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), _NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


<img src="Estimitor.webp" style="width:400px;height:200px;">

构造model_fn

首先我们需要了解iris神经网络结构：
<img src="model_fn.webp" style="width:400px;height:200px;">

## 输入层 Input Layer

<img src="Input Layer.webp" style="width:400px;height:200px;">

<img src="link.webp" style="width:400px;height:200px;">

<img src="link1.webp" style="width:400px;height:200px;">

## 建立model

In [5]:
def my_model_fn(features,labels,mode,params):
    
    #输入层
    net = tf.feature_column.input_layer(features,params['feature_columns'])
    
    #隐藏层
    for units in params['hidden_units']:
        net = tf.layers.dense(net, units=units, activation=tf.nn.relu)
        
    #输出层
    logits = tf.layers.dense(net,params['n_classes'],activation=None)
    
    #预测
    predicted_classes = tf.argmax(logits,1) #预测的结果中最大值即种类
    if mode == tf.estimator.ModeKeys.PREDICT:
        predictions = {
            'class_ids':predicted_classes[:,tf.newaxis],#[1 2 3 4]-->[[1],[2],[3],[4]]
            'logits':logits,#[-1.3,2.6,1.9]
            'probabilities':tf.nn.softmax(logits)#把[-1.3,2.6,-0.9]规则化到0~1范围,表示可能性
        }
        #Ops和对象从`model_fn`返回并传递给`Estimator`。
        return tf.estimator.EstimatorSpec(mode,predictions=predictions)
    
    #损失函数
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels,logits=logits)
    
    #训练
    if mode == tf.estimator.ModeKeys.TRAIN:
        #优化器
        optimizer = tf.train.AdagradOptimizer(learning_rate=0.1)
        train_op = optimizer.minimize(loss,global_step=tf.train.get_global_step())
        return  tf.estimator.EstimatorSpec(mode,loss=loss,train_op=train_op)
    
    #评估
    if mode == tf.estimator.ModeKeys.EVAL:
        accuracy = tf.metrics.accuracy(labels=labels,
                                       predictions=predicted_classes,
                                   name='acc_op')
        metrics = {'accuracy':accuracy}
        #为图表统计使用
        tf.summary.scalar('accuracy',accuracy[1])
        return tf.estimator.EstimatorSpec(mode, loss=loss, eval_metric_ops=metrics)

## 创建分类器

In [ ]:
#修改创建估算分类器的代码设置model_dir模型保存与自动恢复,并设定日志打印
tf.logging.set_verbosity(tf.logging.INFO)
models_path=os.path.join(os.getcwd(),'mymodels/')

#创建自定义分类器
classifier = tf.estimator.Estimator(
        model_fn=my_model_fn,
        model_dir=models_path,
        params={
            'feature_columns': feature_columns,
            'hidden_units': [10, 10],
            'n_classes': 3,
        })

## 训练模型

In [7]:
#针对训练的喂食函数
batch_size=100
def train_input_fn(features, labels, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
    dataset = dataset.shuffle(1000).repeat().batch(batch_size) #每次随机调整数据顺序
    return dataset.make_one_shot_iterator().get_next()

#开始训练
classifier.train(
    input_fn=lambda:train_input_fn(train_x, train_y, 100),
    steps=1000)


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into E:\jupyter\tensorflow\Tensorflow官方文档\高级别API\mymodels/model.ckpt.
INFO:tensorflow:loss = 1.0648383, step = 1
INFO:tensorflow:global_step/sec: 624.844
INFO:tensorflow:loss = 0.14669049, step = 101 (0.160 sec)
INFO:tensorflow:global_step/sec: 997.724
INFO:tensorflow:loss = 0.0793827, step = 201 (0.110 sec)
INFO:tensorflow:global_step/sec: 909.808
INFO:tensorflow:loss = 0.0722641, step = 301 (0.100 sec)
INFO:tensorflow:global_step/sec: 908.842
INFO:tensorflow:loss = 0.06607108, step = 401 (0.110 sec)
INFO:tensorflow:global_step/sec: 1002.54
INFO:tensorflow:loss = 0.08030492, step = 501 (0.100 sec)
INFO:tensorflow:global_step/sec: 907.416
INFO:tensorflow:loss = 0.034899995, step = 601 (0.110 sec)
INFO:tensorflow:

## 评价模型

In [8]:
def eval_input_fn(features,labels,batch_size):
    features = dict(features)
    inputs = (features,labels)
    dataset = tf.data.Dataset.from_tensor_slices(inputs)
    dataset = dataset.batch(batch_size)
    
    return dataset.make_one_shot_iterator().get_next()

#开始评估
eval_results = classifier.evaluate(
    input_fn = lambda:eval_input_fn(test_x,test_y,batch_size)
)
print(eval_results)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-09-06-08:20:20
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from E:\jupyter\tensorflow\Tensorflow官方文档\高级别API\mymodels/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-09-06-08:20:20
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.96666664, global_step = 1000, loss = 0.058778085
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1000: E:\jupyter\tensorflow\Tensorflow官方文档\高级别API\mymodels/model.ckpt-1000
{'accuracy': 0.96666664, 'loss': 0.058778085, 'global_step': 1000}


## 进行预测

In [ ]:
for i in range(0,100):
    print('\nPlease enter features: SepalLength,SepalWidth,PetalLength,PetalWidth')
    a,b,c,d = map(float,input().split(','))#捕捉用户输入数字
    predict_x = {
        'SepalLength': [a],
        'SepalWidth': [b],
        'PetalLength': [c],
        'PetalWidth': [d],
    }
    predictions = classifier.predict(input_fn=lambda:eval_input_fn(predict_x,
                                                                   labels=[0,],
                                                                   batch_size=batch_size))
    
    for pred_dict in predictions:
        class_id = pred_dict['class_ids'][0]
        probability = pred_dict['probabilities'][class_id]
        print(SPECIES[class_id],100 * probability)

        


Please enter features: SepalLength,SepalWidth,PetalLength,PetalWidth
1,2,3,4
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from E:\jupyter\tensorflow\Tensorflow官方文档\高级别API\mymodels/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Virginica 99.99525547027588

Please enter features: SepalLength,SepalWidth,PetalLength,PetalWidth
